In [1]:
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


import os
import csv

In [3]:
#activity labels as defined in activity_labels.txt
activity_labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING', 'STAND_TO_SIT', 'SIT_TO_STAND', 'SIT_TO_LIE', 'LIE_TO_SIT', 'STAND_TO_LIE', 'LIE_TO_STAND']
activity_labels = {k:v for k,v in enumerate(activity_labels, start=1)}

In [4]:
#load data
def load_data(path):
    data = pd.read_csv(path, header=None, delim_whitespace=True)
    return data.values

def load_set(path, x, y):
    data = load_data(path+x)
    labels = load_data(path+y)
    return data, labels

train_data, train_labels = load_set('HAPT Data Set/Train/', 'X_train.txt', 'y_train.txt')
test_data, test_labels = load_set('HAPT Data Set/Test/', 'X_test.txt', 'y_test.txt')

print(f'test_data before mod {test_data.shape}')
print(f'train_data before mod {train_data.shape}')

#reshape the data to add a features dimension (features = 1)
#https://stackoverflow.com/questions/43396572/dimension-of-shape-in-conv1d
train_data = np.expand_dims(train_data, axis=2)
test_data = np.expand_dims(test_data, axis=2)

print(f'test_data after mod {test_data.shape}')
print(f'train_data after mod {train_data.shape}')

test_data before mod (3162, 561)
train_data before mod (7767, 561)
test_data after mod (3162, 561, 1)
train_data after mod (7767, 561, 1)


In [5]:
#input shape
timesteps = train_data.shape[1] #561 timesteps
features = train_data.shape[2] #1 feature

In [18]:
model = models.Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu', input_shape=(timesteps,features)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Dropout(0.4))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(len(activity_labels)))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 559, 32)           128       
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 279, 32)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 277, 64)           6208      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 138, 64)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 136, 128)          24704     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 68, 128)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 68, 128)          

In [19]:
# Compiling the model
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [21]:
test_loss,test_acc = model.evaluate(test_data, test_labels, verbose=2)

3162/3162 - 0s - loss: nan - accuracy: 0.2748


In [22]:
pred_outs = model.predict(test_data)
print(pred_outs)

[[ 0.05269026  0.07346023 -0.22502024 ...  0.04949529 -0.06607392
   0.074191  ]
 [ 0.05144247  0.09305222 -0.23343912 ...  0.06686369 -0.06481856
   0.07483329]
 [ 0.06748015  0.09602307 -0.23130397 ...  0.0426321  -0.06311641
   0.08163658]
 ...
 [ 0.02994326  0.08829673 -0.17334718 ...  0.03612465 -0.05851369
   0.05616707]
 [ 0.02959201  0.0987372  -0.19176367 ...  0.03836277 -0.06901488
   0.05187094]
 [ 0.0583299   0.11174752 -0.18409464 ...  0.02578928 -0.06819558
   0.03511424]]


In [23]:
model.fit(train_data, 
          train_labels, 
          epochs=100, 
          validation_data=(test_data, test_labels))

test_loss,test_acc = model.evaluate( test_data, test_labels, verbose=2)

Train on 7767 samples, validate on 3162 samples
Epoch 1/100
7767/7767 [==============================] - 4s 479us/sample - loss: nan - accuracy: 7.7250e-04 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/100
7767/7767 [==============================] - 3s 368us/sample - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/100
7767/7767 [==============================] - 3s 375us/sample - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/100
7767/7767 [==============================] - 3s 374us/sample - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/100
5632/7767 [====================>.........] - ETA: 0s - loss: nan - accuracy: 0.0000e+00

KeyboardInterrupt: 